In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV

In [2]:
sp500 = yf.Ticker("^GSPC").history(period="max")

In [3]:
# Preprocessing
sp500['Tomorrow'] = sp500['Close'].shift(-1)
sp500['Target'] = (sp500['Tomorrow'] > sp500['Close']).astype(int)
sp500 = sp500.loc["1990-01-01":].drop(['Dividends', 'Stock Splits'], axis=1).dropna()

In [4]:
def feature_engineering(data):
    # Simple Moving Averages for different time windows
    for window in [10, 20, 30, 60, 120, 250]:  # Added 20 to the list
        data[f"MA_{window}"] = data['Close'].rolling(window=window).mean()

    # Exponential Moving Averages
    for window in [10, 30, 60, 120, 250]:
        data[f"EMA_{window}"] = data['Close'].ewm(span=window, adjust=False).mean()

    # Money Flow Index (MFI)
    typical_price = (data['Close'] + data['Low'] + data['High']) / 3
    raw_money_flow = typical_price * data['Volume']
    up_flow = raw_money_flow.copy()
    down_flow = raw_money_flow.copy()
    
    up_flow[typical_price <= typical_price.shift(1)] = 0
    down_flow[typical_price > typical_price.shift(1)] = 0

    up_flow_sum = up_flow.rolling(window=14).sum()
    down_flow_sum = down_flow.rolling(window=14).sum()

    money_flow_ratio = up_flow_sum / down_flow_sum
    data['MFI'] = 100 - (100 / (1 + money_flow_ratio))

    # RSI - Relative Strength Index
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # Bollinger Bands
    data['Bollinger_Upper'] = data['MA_20'] + (data['Close'].rolling(20).std() * 2)
    data['Bollinger_Lower'] = data['MA_20'] - (data['Close'].rolling(20).std() * 2)

    # MACD - Moving Average Convergence Divergence
    short_ema = data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = short_ema - long_ema
    data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Drop NaN values generated by rolling/ewm functions
    data.dropna(inplace=True)
    
    # New Feature: Average True Range (ATR)
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift())
    low_close = np.abs(data['Low'] - data['Close'].shift())
    tr = high_low.combine(high_close, max).combine(low_close, max)
    data['ATR'] = tr.rolling(window=14).mean()

    # Lag Features (Example: 5-day lagged Close)
    data['Lagged_Close'] = data['Close'].shift(5)

    # Normalize Features
    scaler = StandardScaler()
    feature_cols = [col for col in data.columns if col not in ['Tomorrow', 'Target']]
    data[feature_cols] = scaler.fit_transform(data[feature_cols])

    data.dropna(inplace=True)
    return data

In [5]:
sp500 = feature_engineering(sp500)

In [6]:
sp500

,Open,High,Low,Close,Volume,Tomorrow,Target,MA_10,MA_20,MA_30,...,EMA_120,EMA_250,MFI,RSI,Bollinger_Upper,Bollinger_Lower,MACD,Signal_Line,ATR,Lagged_Close
Date,,,,,,,,,,,,,,,,,,,,,
1991-01-15 00:00:00-05:00,-1.208036,-1.209258,-1.206045,-1.207015,-1.319045,316.170013,1,-1.204678,-1.199006,-1.197745,...,-1.199608,-1.193413,-2.009356,-1.903463,-1.192731,-1.204006,-0.297780,-0.231643,-0.891899,-1.206872
1991-01-16 00:00:00-05:00,-1.206891,-1.206308,-1.205023,-1.204761,-1.305646,327.970001,1,-1.205629,-1.199463,-1.197991,...,-1.199730,-1.193536,-1.864494,-1.692381,-1.192948,-1.204719,-0.294020,-0.248137,-0.883826,-1.210029
1991-01-17 00:00:00-05:00,-1.204563,-1.196173,-1.201948,-1.193862,-1.204982,332.230011,1,-1.205068,-1.199559,-1.197940,...,-1.199666,-1.193567,-0.978166,-0.335987,-1.193207,-1.204641,-0.246014,-0.251019,-0.850840,-1.207215
1991-01-18 00:00:00-05:00,-1.193773,-1.192259,-1.191886,-1.189928,-1.255341,331.059998,0,-1.204029,-1.199465,-1.197869,...,-1.199536,-1.193564,-0.390421,-0.075319,-1.192898,-1.204777,-0.191752,-0.241668,-0.837374,-1.206567
1991-01-21 00:00:00-05:00,-1.189801,-1.192259,-1.190223,-1.191008,-1.304702,328.309998,0,-1.202583,-1.199422,-1.197807,...,-1.199426,-1.193571,-0.352680,-0.255278,-1.192765,-1.204830,-0.153268,-0.225919,-0.835062,-1.209103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-20 00:00:00-05:00,2.904861,2.892714,2.868775,2.842762,0.912959,4746.750000,1,2.835074,2.788793,2.749130,...,2.680240,2.676063,0.886011,0.819521,2.782648,2.791378,3.403938,3.385981,1.031978,2.858918
2023-12-21 00:00:00-05:00,2.867504,2.865792,2.878558,2.887466,0.492812,4754.629883,1,2.849992,2.797613,2.760410,...,2.685321,2.679399,0.856419,0.932352,2.797118,2.794137,3.413066,3.442354,1.052817,2.870452
2023-12-22 00:00:00-05:00,2.894875,2.888056,2.904962,2.894744,0.283099,4774.750000,1,2.863901,2.806672,2.773032,...,2.690441,2.682770,1.281615,1.270218,2.810748,2.798291,3.408546,3.486481,1.009742,2.870119


In [7]:
# Split data into train and test sets
train_data, test_data = train_test_split(sp500, test_size=0.2, random_state=42)

In [8]:
# Selecting Features and Target
features = [col for col in sp500.columns if col not in ['Tomorrow', 'Target']]
target = 'Target'

In [9]:
# Model Training
model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=42)
model.fit(train_data[features], train_data[target])

RandomForestClassifier(min_samples_split=100, random_state=42)

In [10]:
# Making Predictions for the RandomForestClassifier as benchmark
predictions = model.predict(test_data[features])
precision = precision_score(test_data[target], predictions)
print(f"Precision Score: {precision}")

Precision Score: 0.5588235294117647


In [11]:
def train_and_tune_model(train_data, features, target):
    # Model Definition
    rf_model = RandomForestClassifier(random_state=42)
    gb_model = GradientBoostingClassifier(random_state=42)
    svc_model = SVC(kernel='rbf', probability=True, random_state=42)

    # Feature Selection using Recursive Feature Elimination with Cross-Validation (RFECV)
    selector = RFECV(estimator=rf_model, step=1, cv=5, scoring='accuracy')
    selector.fit(train_data[features], train_data[target])
    selected_features = [f for f, s in zip(features, selector.support_) if s]

    # Hyperparameter Tuning using GridSearchCV (Example for RandomForest)
    param_grid = {'n_estimators': [100, 200], 'max_depth': [5, 10]}
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='accuracy')
    grid_search.fit(train_data[selected_features], train_data[target])
    tuned_rf_model = grid_search.best_estimator_

    # Create and Train the Ensemble Model
    ensemble_model = VotingClassifier(
        estimators=[('rf', tuned_rf_model), ('gb', gb_model), ('svc', svc_model)],
        voting='soft'
    )
    ensemble_model.fit(train_data[selected_features], train_data[target])

    return ensemble_model, selected_features

In [12]:
ensemble_model, selected_features = train_and_tune_model(train_data, features, target)

In [13]:
# Example of a backtesting function (simplified version)
def backtest(data, model, features):
    predictions = []
    for i in range(250, len(data), 250):
        train = data.iloc[:i]
        test = data.iloc[i:i + 250]
        model.fit(train[features], train[target])
        preds = model.predict(test[features])
        predictions.extend(preds)
    return predictions

In [14]:
svc_model = SVC(kernel='rbf')  # Using RBF kernel for higher dimensions
svc_model.fit(train_data[features], train_data[target])

SVC()

In [15]:
# Implementing Kelly Criterion for position sizing
def kelly_criterion(prob_win, payoff_win):
    return prob_win - (1 - prob_win) / payoff_win


In [18]:
def enhanced_backtest(data, model, features, target, time_stop=10):
    # Backtesting with Kelly Criterion

    predictions = []
    trade_durations = []
    pnl = []

    for i in range(250, len(data), 250):
        train = data.iloc[:i]
        test = data.iloc[i:i + 250]

        model.fit(train[features], train[target])
        preds = model.predict_proba(test[features])[:, 1]
        final_preds = (preds > 0.6).astype(int)

        for j, pred in enumerate(final_preds):
            entry_price = test.iloc[j]['Close']
            exit_price = test.iloc[min(j + time_stop, len(final_preds) - 1)]['Close']
            duration = min(j + time_stop, len(final_preds) - 1) - j
            trade_pnl = (exit_price - entry_price) if pred == 1 else (entry_price - exit_price)
            pnl.append(trade_pnl)
            trade_durations.append(duration)
            predictions.append(pred)

    pnl = np.array(pnl)
    win_rate = np.mean(pnl > 0)
    win_loss_ratio = np.sum(pnl[pnl > 0]) / -np.sum(pnl[pnl < 0])
    kelly_fraction = max(min(win_rate - (1 - win_rate) / win_loss_ratio, 0.5), 0)  # Limiting range

    # Align predictions with dataset
    aligned_predictions = [None] * len(data)
    prediction_index = 0
    for i in range(250, len(data), 250):
        for j in range(i, min(i + 250, len(data))):
            if prediction_index < len(predictions):
                aligned_predictions[j] = predictions[prediction_index]
                prediction_index += 1

    return aligned_predictions, pnl, trade_durations, kelly_fraction

In [20]:
# Running the backtest
backtest_results = enhanced_backtest(sp500, ensemble_model, selected_features, target)
backtest_predictions, pnl, trade_durations, kelly_fraction = backtest_results

# Removing None values from predictions for precision score calculation
valid_predictions = [pred for pred in backtest_predictions if pred is not None]
valid_targets = sp500[target].iloc[len(sp500[target]) - len(valid_predictions):]

# Analyzing the results
print(f"Backtest Precision: {precision_score(valid_targets, valid_predictions)}")
print(f"Average P&L per trade: {np.mean(pnl)}")
print(f"Average trade duration: {np.mean(trade_durations)} days")
print(f"Kelly Fraction: {kelly_fraction}")

Backtest Precision: 0.5410557184750733
Average P&L per trade: -0.003934294560805422
Average trade duration: 9.774562166190535 days
Kelly Fraction: 0
